In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: kodikarthik21 (use `wandb login --relogin` to force relogin)


True

In [ ]:
!unzip '/content/drive/MyDrive/nature_12K'

Archive:  /content/drive/MyDrive/nature_12K.zip
replace inaturalist_12K/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

#Question 1 (5 Marks)



Build a small CNN model consisting of  5 convolution layers. Each convolution layer would be followed by a ReLU activation and a max pooling layer. Here is sample code for building one such conv-relu-maxpool block in keras. 

After 5 such conv-relu-maxpool blocks of  layers you should have one dense layer followed by the output layer containing 10 neurons (1 for each of the 10 classes). The input layer should be compatible with the images in the iNaturalist dataset.






In [ ]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
tf.test.gpu_device_name()

''

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/inaturalist_12K/train', 
                                                                    validation_split=0.1, seed = 123, subset = "training")

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/inaturalist_12K/train', 
                                                                    validation_split=0.1, seed = 123, subset = "validation")

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
class_names = train_dataset.class_names
for images, labels in train_dataset.take(1):
    for i in range(32):
        ax = plt.subplot(6, 6, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
def sweep_fn(): 
#---------HYPERPARAMETER DEFINITIONS--------------------------------------------
  hyperparameter_defaults = dict(
        max_epoch = 10,
        num_filters = 32,
        filter_org = 'same',
        data_augmentation = 'N',
        dropout = 0,
        batch_normalization = 'N',
        optimizer = 'adam'
      )
  learning_rate = 1e-3
  decay_rate = learning_rate / epochs
  momentum = 0.8
  if config.optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'nadam':
    optimizer = tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.rmsprop(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'nag':
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate, decay=decay_rate, nesterov=True)

#-----------MODEL FRAMEWORK-----------------------------------------------------
  wandb.init(project="Assignment02", config=hyperparameter_defaults)
  config = wandb.config
  nf = config.num_filters

  model = Sequential()
  model.add(Conv2D(nf, (3, 3), input_shape=(256, 256, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  for i in range(1,5):
    if config.filter_org == 'double':
      nf = nf*2
    elif config.filter_org == 'half':
      nf = nf/2
    model.add(Conv2D(nf, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(64))
  model.add(Dense(10))
  model.add(Activation('softmax'))
 
#-------------------------------------------------------------------------------
  model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics='accuracy')
  model.fit(train_dataset, validation_data = validation_dataset, batch_size = 32, epochs = config.max_epoch)
  callbacks=[WandbCallback()] 
  model.save('iNaturalist_CNN')
  model.summary()   
  wandb.run.finish()

In [ ]:
import wandb

sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment02",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "max_epoch": {
            "values": [10]
        },
        "num_filters": {
            "values":[32,64,128]
        }, 
        "filter_org": {
            "values":['same','double','half']
        },
        "data_augmentation":{
            "values":['Y',"N"]
        },
        "dropout":{
            "values":[0, 0.2,0.3,0.4]
        },
        "batch_normalization": {
            "values":['Y',"N"]
        },  
        "optimizer": {
            "values":['adam', 'nadam','rmsprop','nag']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: oayxwoty
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/oayxwoty


In [ ]:
wandb.agent(sweep_id, function=sweep_fn)

In [ ]:
sweep_fn()

wandb: WARNING Ignored wandb.init() arg project when running a sweep
